## Loading and Setup

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import time
import pickle
# Filter warnings
from warnings import filterwarnings
filterwarnings('ignore')

/Users/rachelliao/anaconda3/envs/ensemble/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Load the datasets
train_df = pd.read_parquet('dataset/train_dataset.parquet')
test_df = pd.read_parquet('dataset/test_dataset.parquet')

In [3]:
# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Assessment

### Train Data

In [4]:
# Display the first 5 rows of train data to get an overview of the data
train_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
0,0.411765,0.411765,0.411765,0.0,0.162653,0.338074,0.015893,0.089071,0.000000,0.108108,0.107527,0.0,0.140625,0.043478,0.013267,0.447506,0.065421,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.676471,0.676471,0.676471,1.0,0.577524,0.447021,0.026730,0.253851,0.066667,0.027027,0.026882,0.0,0.140625,0.000000,0.031700,0.483246,0.168224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.872727,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.411765,0.411765,0.411765,0.0,0.189269,0.341412,0.046797,0.128641,0.000000,0.270270,0.268817,0.6,0.281250,0.000000,0.040715,0.294862,0.261682,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.316176,0.316176,0.316176,0.0,0.281791,0.271637,0.007866,0.218004,0.000000,0.054054,0.053763,0.0,0.062500,0.000000,0.020613,0.588235,0.196262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.690909,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.264706,0.264706,0.264706,0.0,0.224757,0.222903,0.011880,0.087391,0.000000,0.108108,0.107527,0.2,0.046875,0.000000,0.008994,0.446761,0.056075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [5]:
# Display the last 5 rows of the train data to get an overview of the data  
train_df.tail()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
27391,0.284751,0.284751,0.284751,0.0,0.249261,0.241881,0.018642,0.168478,0.056254,0.061236,0.060906,0.084381,0.176027,0.025135,0.015186,0.465594,0.205759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.596017,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0
27392,0.249576,0.249576,0.249576,0.0,0.081960,0.198987,0.029849,0.157311,0.000000,0.282614,0.281095,0.000000,0.187049,0.000000,0.063611,0.361683,0.312410,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.890909,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.257214,0.0,0.0
27393,0.289427,0.289427,0.289427,0.0,0.332066,0.253502,0.014361,0.219024,0.047125,0.079451,0.079024,0.470687,0.114897,0.015367,0.016247,0.534641,0.116580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.864108,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0
27394,0.712357,0.712357,0.712357,1.0,0.544149,0.462675,0.025819,0.193246,0.000000,0.011559,0.011497,0.157233,0.156250,0.000000,0.010436,0.574177,0.154235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.779188,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0
27395,0.303379,0.303379,0.302538,0.0,0.133925,0.246652,0.022270,0.212239,0.000000,0.158281,0.157430,0.200000,0.305851,0.000000,0.051815,0.439693,0.284475,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.836412,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0


In [6]:
# Randomly sample 15 rows from the train data
train_df.sample(15)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
17038,0.773956,0.773956,0.773956,1.0,0.529785,0.499196,0.029846,0.143659,0.000000,0.263936,0.262517,0.20000,0.095581,0.000000,0.028825,0.576106,0.075861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.743469,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15993,0.966912,0.966912,0.966912,1.0,0.799324,0.716476,0.027894,0.039133,0.000000,0.216216,0.215054,0.00000,0.140625,0.000000,0.019320,0.238273,0.093458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5837,1.000000,1.000000,1.000000,0.0,0.249261,0.845067,0.096966,0.064248,0.200000,0.027027,0.026882,0.00000,0.171875,0.086957,0.049724,0.618019,0.280374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.490909,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4456,0.794118,0.794118,0.794118,1.0,0.493029,0.502364,0.030743,0.100095,0.000000,0.108108,0.107527,0.00000,0.109375,0.000000,0.034908,0.678332,0.112150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.745455,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
5128,0.264706,0.264706,0.264706,0.0,0.162653,0.217692,0.012610,0.144338,0.000000,0.108108,0.107527,0.00000,0.265625,0.000000,0.013945,0.555473,0.271028,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
16188,0.304412,0.304412,0.304412,0.0,0.476130,0.280798,0.005943,0.221946,0.066667,0.000000,0.000000,0.00000,0.140625,0.000000,0.015754,0.566642,0.149533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
13046,0.588235,0.588235,0.588235,1.0,0.281791,0.329645,0.022716,0.236986,0.000000,0.081081,0.080645,0.00000,0.218750,0.000000,0.034015,0.640357,0.392523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.781818,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4817,0.117647,0.117647,0.117647,0.0,0.189269,0.098428,0.008669,0.034460,0.000000,0.054054,0.053763,0.00000,0.078125,0.043478,0.002642,0.208488,0.065421,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.763636,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
6142,0.135294,0.135294,0.135294,0.0,0.224757,0.114680,0.009472,0.132803,0.000000,0.189189,0.188172,0.00000,0.046875,0.000000,0.006924,0.710350,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

This quick check seems to show that the data is loaded correctly now.

In [7]:
# Get information about the train data
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27396 entries, 0 to 27395
Data columns (total 70 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            27396 non-null  float64
 1   funded_amnt                          27396 non-null  float64
 2   funded_amnt_inv                      27396 non-null  float64
 3   term                                 27396 non-null  float64
 4   int_rate                             27396 non-null  float64
 5   installment                          27396 non-null  float64
 6   annual_inc                           27396 non-null  float64
 7   dti                                  27396 non-null  float64
 8   delinq_2yrs                          27396 non-null  float64
 9   fico_range_low                       27396 non-null  float64
 10  fico_range_high                      27396 non-null  float64
 11  inq_last_6mths              

In [8]:
# Get a statistical summary of the train data
train_df.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
count,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000,27396.000000
mean,0.424273,0.424273,0.424073,0.352774,0.330396,0.314067,0.027485,0.146731,0.021904,0.167929,0.167027,0.133697,0.177501,0.010817,0.026224,0.393520,0.201764,0.018284,0.018475,0.023890,0.035890,0.042898,0.050308,0.054943,0.060753,0.057161,0.067602,0.059236,0.062568,0.070559,0.052999,0.041858,0.036884,0.032376,0.033136,0.023331,0.024551,0.019607,0.019980,0.017981,0.012334,0.007828,0.007836,0.006933,0.002896,0.003695,0.001488,0.002926,0.000717,0.000616,0.000308,0.773537,0.106557,0.422449,0.416241,0.318917,0.230676,0.598559,0.049941,0.003417,0.019556,0.009275,0.006070,0.055252,0.000519,0.012502,0.005616,0.876642,0.006244,0.500000
std,0.248122,0.248122,0.247986,0.473644,0.182873,0.180160,0.020162,0.060421,0.053685,0.150542,0.149735,0.173767,0.085944,0.026039,0.031657,0.168806,0.107976,0.133229,0.134243,0.151951,0.185237,0.202224,0.217915,0.227365,0.238299,0.231318,0.250408,0.235173,0.241693,0.255254,0.222993,0.199274,0.187081,0.175823,0.177478,0.149504,0.153835,0.137496,0.138837,0.131720,0.109136,0.085850,0.087245,0.081333,0.052201,0.058780,0.035920,0.050973,0.024969,0.022402,0.016111,0.121925,0.301880,0.488733,0.489811,0.462742,0.419931,0.486635,0.215498,0.055917,0.134792,0.093885,0.075333,0.225907,0.021822,0.106618,0.072358,0.319369,0.074615,0.500009
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.235294,0.235294,0.235294,0.000000,0.187579,0.183456,0.016696,0.101850,0.000000,0.054054,0.053763,0.000000,0.119353,0.000000,0.010289,0.270505,0.123071,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.709091,0.000000,0.000000,0.

### Test Data

In [9]:
# Display the first 5 rows of the test data to get an overview of the data
test_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
0,0.794118,0.794118,0.794118,0.0,0.108576,0.638507,0.277573,0.023582,0.000000,0.243243,0.241935,0.0,0.125000,0.0,0.098225,0.588235,0.149533,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.745455,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.464706,0.464706,0.464706,0.0,0.281791,0.398295,0.013887,0.167044,0.000000,0.162162,0.161290,0.0,0.156250,0.0,0.009498,0.423678,0.196262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.818182,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.676471,0.676471,0.676471,1.0,0.133925,0.348722,0.056831,0.173031,0.066667,0.135135,0.134409,0.0,0.171875,0.0,0.340650,0.681310,0.261682,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.618182,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.676471,0.676471,0.676471,0.0,0.000000,0.522777,0.046797,0.179163,0.000000,0.459459,0.456989,0.0,0.187500,0.0,0.111296,0.453462,0.214953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.581818,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.676471,0.676471,0.675000,1.0,0.249261,0.373018,0.022716,0.138424,0.000000,0.297297,0.295699,0.0,0.125000,0.0,0.009972,0.331348,0.084112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.909091,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
# Display the last 5 rows of the test data to get an overview of the data
test_df.tail()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
4162,0.308824,0.308824,0.308824,0.0,0.260245,0.263254,0.006662,0.148938,0.000000,0.378378,0.376344,0.0,0.078125,0.043478,0.000811,0.037975,0.056075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.909091,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4163,0.157353,0.157353,0.157353,0.0,0.386988,0.141966,0.006020,0.077097,0.000000,0.270270,0.268817,0.0,0.093750,0.000000,0.011396,0.270290,0.056075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.909091,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4164,0.558824,0.558824,0.557353,0.0,0.281791,0.478552,0.044790,0.198365,0.000000,0.162162,0.161290,0.0,0.109375,0.000000,0.281839,0.662695,0.074766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.836364,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4165,0.308824,0.308824,0.308824,0.0,0.162653,0.253806,0.015893,0.125210,0.066667,0.027027,0.026882,0.0,0.046875,0.000000,0.012660,0.426657,0.074766,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4166,0.535294,0.535294,0.535294,0.0,0.000000,0.413611,0.026730,0.022268,0.000000,0.243243,0.241935,0.0,0.156250,0.000000,0.004625,0.082651,0.168224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.781818,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [11]:
# Randomly sample 15 rows from the test data
test_df.sample(15)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
4064,0.558824,0.558824,0.558824,1.0,0.386988,0.332160,0.036764,0.248449,0.000000,0.027027,0.026882,0.2,0.203125,0.000000,0.056770,0.510797,0.271028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.818182,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3743,0.705882,0.705882,0.705882,0.0,0.352767,0.619204,0.024723,0.132584,0.000000,0.189189,0.188172,0.0,0.093750,0.043478,0.007903,0.401340,0.046729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.854545,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2979,0.205882,0.205882,0.205882,0.0,0.129278,0.167326,0.022716,0.134701,0.000000,0.189189,0.188172,0.0,0.203125,0.000000,0.029933,0.323902,0.186916,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.890909,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3277,0.117647,0.117647,0.117647,0.0,0.000000,0.090656,0.027733,0.129225,0.000000,0.135135,0.134409,0.0,0.265625,0.043478,0.009439,0.177215,0.317757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.636364,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3971,0.411765,0.411765,0.411765,1.0,0.474440,0.255740,0.033553,0.192451,0.000000,0.135135,0.134409,0.0,0.187500,0.000000,0.025365,0.536113,0.177570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.818182,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1237,1.000000,1.000000,1.000000,1.0,0.615547,0.675626,0.063413,0.043732,0.000000,0.000000,0.000000,0.4,0.062500,0.043478,0.014092,0.320179,0.130841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
743,0.176471,0.176471,0.176471,0.0,0.187579,0.146928,0.007064,0.223480,0.000000,0.243243,0.241935,0.4,0.234375,0.000000,0.008767,0.327625,0.177570,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.927273,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
696,0.464706,0.464706,0.464706,0.0,0.366286,0.410514,0.035559,0.122509,0.133333,0.054054,0.053763,0.2,0.203125,0.043478,0.023556,0.379747,0.224299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.672727,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1265,0.205882,0.205882,0.205882,0.0,0.133925,0.167636,0.013485,0.157480,0.000000,0.189189,0.188172,0.0,0.093750,0.043478,0.007262,0.250186,0.149533,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
21,0.676471,0.676471,0.67647

This quick check seems to show that the data is loaded correctly now.

In [12]:
# Get information about the test data
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4167 entries, 0 to 4166
Data columns (total 70 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            4167 non-null   float64
 1   funded_amnt                          4167 non-null   float64
 2   funded_amnt_inv                      4167 non-null   float64
 3   term                                 4167 non-null   float64
 4   int_rate                             4167 non-null   float64
 5   installment                          4167 non-null   float64
 6   annual_inc                           4167 non-null   float64
 7   dti                                  4167 non-null   float64
 8   delinq_2yrs                          4167 non-null   float64
 9   fico_range_low                       4167 non-null   float64
 10  fico_range_high                      4167 non-null   float64
 11  inq_last_6mths                

In [13]:
# Get a statistical summary of the test data
test_df.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
count,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.00000,4167.00000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000
mean,0.414134,0.414134,0.413938,0.309575,0.293546,0.307405,0.028483,0.140749,0.023390,0.188975,0.187959,0.122150,0.174196,0.010110,0.026815,0.385174,0.200099,0.023998,0.025198,0.034317,0.052316,0.049916,0.057835,0.066715,0.066955,0.057115,0.069834,0.059755,0.056635,0.057355,0.046076,0.036957,0.029758,0.025198,0.027118,0.022318,0.017519,0.017519,0.015119,0.007919,0.010079,0.006479,0.00504,0.00408,0.001920,0.001920,0.001680,0.001680,0.000720,0.000480,0.000480,0.762345,0.108711,0.403408,0.401488,0.284617,0.249100,0.575714,0.053756,0.005280,0.024238,0.010319,0.006479,0.050396,0.000480,0.008399,0.005760,0.868731,0.005280,0.821934
std,0.256075,0.256075,0.255897,0.462374,0.179165,0.187904,0.032038,0.064629,0.060958,0.167534,0.166633,0.178988,0.089411,0.026739,0.032626,0.178535,0.111500,0.153061,0.156745,0.182065,0.222690,0.217798,0.233460,0.249557,0.249973,0.232091,0.254898,0.237061,0.231172,0.232548,0.209676,0.188679,0.169938,0.156745,0.162446,0.147734,0.131209,0.131209,0.122040,0.088648,0.099900,0.080244,0.07082,0.06375,0.043779,0.043779,0.040957,0.040957,0.026825,0.021905,0.021905,0.134950,0.311314,0.490640,0.490258,0.451286,0.432544,0.494293,0.225562,0.072477,0.153806,0.101070,0.080244,0.218787,0.021905,0.091273,0.075682,0.337735,0.072477,0.382614
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.205882,0.205882,0.205882,0.000000,0.162653,0.170132,0.015963,0.092648,0.000000,0.054054,0.053763,0.000000,0.109375,0.000000,0.009791,0.253909,0.121495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.690909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

## Split into Training and Validation Datasets

In [14]:
# Separate features and target variable from the training dataset
X_train_full = train_df.drop(columns=['loan_status'])
y_train_full = train_df['loan_status']

# Split the training dataset into training and validation subsets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Separate features and target variable from the test dataset
X_test = test_df.drop(columns=['loan_status'])
y_test = test_df['loan_status']

# Check the shapes of the datasets
X_train.shape, X_val.shape, X_test.shape

((21916, 69), (5480, 69), (4167, 69))

## Baseline Modelling

### Logistic Regression

In [15]:
# Initialize the model
log_reg = LogisticRegression(random_state=42)

# Train the model
log_reg.fit(X_train, y_train)

# Make predictions
preds = log_reg.predict(X_val)

# Compute metrics
accuracy_lg = accuracy_score(y_val, preds)
precision_lg = precision_score(y_val, preds)
recall_lg = recall_score(y_val, preds)
f1_lg = f1_score(y_val, preds)
conf_mat_lg = confusion_matrix(y_val, preds)
clf_report_lg = classification_report(y_val, preds)

# Print metrics
print("Logistic Regression Metrics:")
print(f"Accuracy: {accuracy_lg}")
print(f"Precision: {precision_lg}")
print(f"Recall: {recall_lg}")
print(f"F1 Score: {f1_lg}")
print("Confusion Matrix:")
print(conf_mat_lg)
print("Classification Report:")
print(clf_report_lg)

Logistic Regression Metrics:
Accuracy: 0.6618613138686131
Precision: 0.6829846032372681
Recall: 0.6223021582733813
F1 Score: 0.6512328251458686
Confusion Matrix:
[[1897  803]
 [1050 1730]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.64      0.70      0.67      2700
         1.0       0.68      0.62      0.65      2780

    accuracy                           0.66      5480
   macro avg       0.66      0.66      0.66      5480
weighted avg       0.66      0.66      0.66      5480



### Support Vector Machine

In [16]:
# Initialize the model
svm = SVC(random_state=42)

# Train the model
svm.fit(X_train, y_train)

# Make predictions
preds = svm.predict(X_val)

# Compute metrics
accuracy_svm = accuracy_score(y_val, preds)
precision_svm = precision_score(y_val, preds)
recall_svm = recall_score(y_val, preds)
f1_svm = f1_score(y_val, preds)
conf_mat_svm = confusion_matrix(y_val, preds)
clf_report_svm = classification_report(y_val, preds)

# Print metrics
print("Support Vector Machine Metrics:")
print(f"Accuracy: {accuracy_svm}")
print(f"Precision: {precision_svm}")
print(f"Recall: {recall_svm}")
print(f"F1 Score: {f1_svm}")
print("Confusion Matrix:")
print(conf_mat_svm)
print("Classification Report:")
print(clf_report_svm)

Support Vector Machine Metrics:
Accuracy: 0.7397810218978103
Precision: 0.7615919629057187
Recall: 0.7089928057553957
F1 Score: 0.7343517138599105
Confusion Matrix:
[[2083  617]
 [ 809 1971]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.72      0.77      0.74      2700
         1.0       0.76      0.71      0.73      2780

    accuracy                           0.74      5480
   macro avg       0.74      0.74      0.74      5480
weighted avg       0.74      0.74      0.74      5480



### Decision Tree

In [17]:
# Initialize the model
dec_tree = DecisionTreeClassifier(random_state=42)

# Train the model
dec_tree.fit(X_train, y_train)

# Make predictions
preds = dec_tree.predict(X_val)

# Compute metrics
accuracy_dt = accuracy_score(y_val, preds)
precision_dt = precision_score(y_val, preds)
recall_dt = recall_score(y_val, preds)
f1_dt = f1_score(y_val, preds)
conf_mat_dt = confusion_matrix(y_val, preds)
clf_report_dt = classification_report(y_val, preds)

# Print metrics
print("Decision Tree Metrics:")
print(f"Accuracy: {accuracy_dt}")
print(f"Precision: {precision_dt}")
print(f"Recall: {recall_dt}")
print(f"F1 Score: {f1_dt}")
print("Confusion Matrix:")
print(conf_mat_dt)
print("Classification Report:")
print(clf_report_dt)

Decision Tree Metrics:
Accuracy: 0.7952554744525547
Precision: 0.8218167701863354
Recall: 0.7615107913669065
F1 Score: 0.7905153099327857
Confusion Matrix:
[[2241  459]
 [ 663 2117]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.77      0.83      0.80      2700
         1.0       0.82      0.76      0.79      2780

    accuracy                           0.80      5480
   macro avg       0.80      0.80      0.80      5480
weighted avg       0.80      0.80      0.80      5480



### Random Forest

In [18]:
# Initialize the model
rand_forest = RandomForestClassifier(random_state=42)

# Train the model
rand_forest.fit(X_train, y_train)

# Make predictions
preds = rand_forest.predict(X_val)

# Compute metrics
accuracy_rf = accuracy_score(y_val, preds)
precision_rf = precision_score(y_val, preds)
recall_rf = recall_score(y_val, preds)
f1_rf = f1_score(y_val, preds)
conf_mat_rf = confusion_matrix(y_val, preds)
clf_report_rf = classification_report(y_val, preds)

# Print metrics
print("Random Forest Metrics:")
print(f"Accuracy: {accuracy_rf}")
print(f"Precision: {precision_rf}")
print(f"Recall: {recall_rf}")
print(f"F1 Score: {f1_rf}")
print("Confusion Matrix:")
print(conf_mat_rf)
print("Classification Report:")
print(clf_report_rf)

Random Forest Metrics:
Accuracy: 0.8959854014598541
Precision: 0.8718034993270525
Recall: 0.9320143884892086
F1 Score: 0.900904033379694
Confusion Matrix:
[[2319  381]
 [ 189 2591]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.92      0.86      0.89      2700
         1.0       0.87      0.93      0.90      2780

    accuracy                           0.90      5480
   macro avg       0.90      0.90      0.90      5480
weighted avg       0.90      0.90      0.90      5480



### XGBoost

In [19]:
# Initialize the model
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train the model
xgb.fit(X_train, y_train)

# Make predictions
preds = xgb.predict(X_val)

# Compute metrics
accuracy_xgb = accuracy_score(y_val, preds)
precision_xgb = precision_score(y_val, preds)
recall_xgb = recall_score(y_val, preds)
f1_xgb = f1_score(y_val, preds)
conf_mat_xgb = confusion_matrix(y_val, preds)
clf_report_xgb = classification_report(y_val, preds)

# Print metrics
print("XGBoost Metrics:")
print(f"Accuracy: {accuracy_xgb}")
print(f"Precision: {precision_xgb}")
print(f"Recall: {recall_xgb}")
print(f"F1 Score: {f1_xgb}")
print("Confusion Matrix:")
print(conf_mat_xgb)
print("Classification Report:")
print(clf_report_xgb)

[22:47:12] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1593723618214/work/src/learner.cc:480: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


XGBoost Metrics:
Accuracy: 0.8874087591240876
Precision: 0.8384976525821596
Recall: 0.9636690647482015
F1 Score: 0.8967364016736401
Confusion Matrix:
[[2184  516]
 [ 101 2679]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.81      0.88      2700
         1.0       0.84      0.96      0.90      2780

    accuracy                           0.89      5480
   macro avg       0.90      0.89      0.89      5480
weighted avg       0.90      0.89      0.89      5480



### Summary of Baseline Models

In [20]:
# Create an empty DataFrame to store the metrics
metrics_df = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Add the metrics for the baseline models
metrics_df.loc['Logistic Regression'] = [accuracy_lg, precision_lg, recall_lg, f1_lg]
metrics_df.loc['Support Vector Machine'] = [accuracy_svm, precision_svm, recall_svm, f1_svm]
metrics_df.loc['Decision Tree'] = [accuracy_dt, precision_dt, recall_dt, f1_dt]
metrics_df.loc['Random Forest'] = [accuracy_rf, precision_rf, recall_rf, f1_rf]
metrics_df.loc['XGBoost'] = [accuracy_xgb, precision_xgb, recall_xgb, f1_xgb]

# Print the comparison table
print(metrics_df)

                        Accuracy  Precision    Recall  F1 Score
Logistic Regression     0.661861   0.682985  0.622302  0.651233
Support Vector Machine  0.739781   0.761592  0.708993  0.734352
Decision Tree           0.795255   0.821817  0.761511  0.790515
Random Forest           0.895985   0.871803  0.932014  0.900904
XGBoost                 0.887409   0.838498  0.963669  0.896736
